<a href="https://colab.research.google.com/github/preeti13456/Automating/blob/main/Automating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import time
import random

In [6]:
def get_instagram_profile(username):
    """Fetches and parses data from an Instagram profile.

    Args:
        username (str): The username of the Instagram profile to scrape.

    Returns:
        dict: A dictionary containing profile information or None if unsuccessful.

    Raises:
        requests.exceptions.RequestException: If there's an issue with the request.
    """

    url = f"https://www.instagram.com/{username}/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        follower_count_element = soup.find('span', class_='g41i0')
        follower_count = follower_count_element.text.strip() if follower_count_element else None

        # Extract engagement rate (based on recent posts)
        posts = soup.find_all('div', class_='e8sjw')
        likes_sum = 0
        comments_sum = 0
        post_count = 0

        for post in posts:
            likes_element = post.find('span', class_='g41i0')
            comments_element = post.find('span', class_='g41i0')

            if likes_element and comments_element:
                likes_sum += int(likes_element.text.strip())
                comments_sum += int(comments_element.text.strip())
                post_count += 1

        # Check for valid values before calculating engagement rate
        if follower_count and post_count:
            engagement_rate = (likes_sum + comments_sum) / (follower_count * post_count) * 100
        else:
            engagement_rate = None

        return {'username': username, 'follower_count': follower_count, 'engagement_rate': engagement_rate}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching profile for {username}: {e}")
        return None


def collect_influencer_data(usernames):
    """Collects data for a list of Instagram usernames.

    Args:
        usernames (list): A list of Instagram usernames.

    Returns:
        list: A list of dictionaries containing profile data or None values.
    """

    data = []
    for username in usernames:
        profile_data = get_instagram_profile(username)
        if profile_data:
            data.append(profile_data)
        # Add a random delay to avoid rate limiting
        time.sleep(random.uniform(1, 3))
    return data


# Example usage
usernames = ['username1', 'username2']
data = collect_influencer_data(usernames)

# Save data to a CSV file (assuming pandas is installed)
if data:
    df = pd.DataFrame(data)
    df.to_csv('influencer_data.csv', index=False)